<a href="https://colab.research.google.com/github/alexstar55/CV_Notes/blob/master/Dockerfile%E6%A8%A1%E6%9D%BF%E7%A4%BA%E4%BE%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 简介  
- 为什么用Dockerfile？因为它生成镜像方便，方便订制，也就是为部署提供便利。  
- 这个东西有些格式，下面以这次部署做的Dockerfile为例，可以当成模板，修改部署的环境（cuda, cudnn，各种包）然后生成自己的镜像。

## 基本流程  
- 基础镜像（Cuda, Cudnn, ubuntu）
- 基础安装（apt-get）
- python环境建立（这里用miniconda建，因为方（省）便（心））
- 安装各种包
- 其他设置与清理（时区、软连接、清理等）  
下面分别说明按自己的需要构建时，要改哪些东西：

### 基础镜像（Cuda, Cudnn, ubuntu）  
```
FROM nvidia/cuda:9.0-devel-ubuntu16.04
ENV CUDNN_VERSION=7.4.1.5
```  
通过设定这两行，来指定ubuntu、cuda、cudnn版本。  
FROM跟着基础镜像名，通过英伟达的镜像可以指定cuda9.0，ubuntu16.04。对于cudnn，英伟达的9.0-cudnn7-devel-ubuntu16.04镜像cudnn版本是7.0，如果想用别的版本cudnn，就用这个例子的方法即可。  
ENV CUDNN_VERSION=7.4.1.5就是设定的版本。

### 基础安装（apt-get）  
```
RUN apt-get update && apt-get install -y --no-install-recommends \
        build-essential \
        apt-utils curl g++ \
    cmake \
        git \
        wget \
        libopencv-dev \
        libsnappy-dev \
        python-dev \
        python-pip \
        tzdata \
        vim
```
这里是apt-get安装一些基础的东西，一般不用改。当然如果有需求，比如把wget换成aria2，或者前面换个下载源，都可以。

### python环境建立（这里用miniconda建，因为方（省）便（心））  
使用miniconda改python版本很方便：
```
# Create a Python 3.6 environment
RUN /miniconda/bin/conda install -y conda-build \
 && /miniconda/bin/conda create -y --name py367 python=3.6.7 \
 && /miniconda/bin/conda clean -ya
 ```  
 这里通过python=3.6.7设置python版本，可以按需改

### 安装各种包  
这个没啥，我们已经安了pip，直接pip install:
```
RUN pip install numpy==1.15.4 \
        pandas==0.20.3 \
        imgaug==0.2.9 \
        opencv_python==4.0.0.21 \
        h5py==2.7.0 \
        scikit_image==0.13.0 \
        scipy==0.19.1 \
        requests==2.18.4 \
        Pillow==6.1.0 \
        barcode_generator==0.1rc15 \
        easywebdav==1.2.0 \
        paramiko==2.6.0 \
        pydrive==1.3.1 \
        schedule==0.6.0 \
        wget \
        lxml && \
    pip install tensorflow-gpu==1.12.0 \
        Keras==2.2.4 \
        && \ 
```
注意，pip install各包直接不用&&，但是顺序很重要，被依赖的要放在前面安。这里单独把tensorflow放在后面安，确保前面的已安完了。
      

### 其他设置与清理（时区、软连接、清理等）  
这里要注意两点：  
- Docker容器默认时间是UTC，如果代码运行时要用北京时间或别的时间，一个办法是生成容器时挂载这个来显示宿主机时间：-v /etc/localtime:/etc/localtime:ro; 另一个方法就是这里的，在Dockerfile里设置时区，
```
ln -sf /usr/share/zoneinfo/Asia/Tokyo /etc/localtime && \
```
这一行设置的是东京时间，如果改成Asia/Shanghai那就是北京时间（我也不知道为什么系统里用上海表示北京时间）  
- 设置cudnn软连接，因为[某种奇怪的原因](https://github.com/tensorflow/tensorflow/issues/6850)，tensorflow在容器里会找不到cudnn。因为cudnn7的文件是cudnn_v7.h而tf只认cudnn.h，所以要加个软连接，  
```
mkdir /usr/lib/x86_64-linux-gnu/include && \
    ln -s /usr/include/x86_64-linux-gnu/cudnn_v7.h /usr/lib/x86_64-linux-gnu/include/cudnn.h && \
```
这里的cudnn是7，如果是别的版本的cudnn，可以先去官网看看对应版本名字是不是cudnn.h，然后按需修改。  
剩下的收尾工作，一般不用改。  
最后EXPOSE了一个端口，这个没什么影响，也可以按需修改。

## 镜像生成  
在Dockerfile所在的目录里运行：  
>docker build -t {dockerhub库名}:{镜像名} .

注意最后是一个“.” 。  
这样生成镜像，然后docker push到相应的docker hub就可以了。

## 相关资料  
- https://zhuanlan.zhihu.com/p/63055844
- https://www.centos.bz/2016/11/running-docker-container-as-daemon/
-https://gitlab.com/nvidia/cuda/blob/ubuntu16.04/9.0/devel/cudnn7/Dockerfile

附完整Dockerfile文件：  
```
FROM nvidia/cuda:9.0-devel-ubuntu16.04
ENV CUDNN_VERSION=7.4.1.5
LABEL com.nvidia.cudnn.version="${CUDNN_VERSION}"
RUN echo 'debconf debconf/frontend select Noninteractive' | debconf-set-selections

RUN apt-get update && apt-get install -y --no-install-recommends \
            libcudnn7=$CUDNN_VERSION-1+cuda9.0 \
            libcudnn7-dev=$CUDNN_VERSION-1+cuda9.0 && \
    apt-mark hold libcudnn7 
# install basics
RUN apt-get update && apt-get install -y --no-install-recommends \
        build-essential \
        apt-utils curl g++ \
	cmake \
        git \
        wget \
        libopencv-dev \
        libsnappy-dev \
        python-dev \
        python-pip \
        tzdata \
        vim
# Install anaconda for python 3.6
# Install Miniconda
RUN curl -so /miniconda.sh https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh \
 && chmod +x /miniconda.sh \
 && /miniconda.sh -b -p /miniconda \
 && rm /miniconda.sh \
 && rm -Rf /var/lib/apt/lists/* \
 && apt-get update
 

ENV PATH=/miniconda/bin:$PATH

# Create a Python 3.6 environment
RUN /miniconda/bin/conda install -y conda-build \
 && /miniconda/bin/conda create -y --name py367 python=3.6.7 \
 && /miniconda/bin/conda clean -ya

ENV CONDA_DEFAULT_ENV=py367
ENV CONDA_PREFIX=/miniconda/envs/$CONDA_DEFAULT_ENV
ENV PATH=$CONDA_PREFIX/bin:$PATH
ENV CONDA_AUTO_UPDATE_CONDA=false
RUN pip install numpy==1.15.4 \
        pandas==0.20.3 \
        imgaug==0.2.9 \
        opencv_python==4.0.0.21 \
        h5py==2.7.0 \
        scikit_image==0.13.0 \
        scipy==0.19.1 \
        requests==2.18.4 \
        Pillow==6.1.0 \
        barcode_generator==0.1rc15 \
        easywebdav==1.2.0 \
        paramiko==2.6.0 \
        pydrive==1.3.1 \
        schedule==0.6.0 \
        wget \
        lxml && \
    pip install tensorflow-gpu==1.12.0 \
        Keras==2.2.4 \
        && \
# ==================================================================
# config & cleanup
# ------------------------------------------------------------------
    # Set timezone
    ln -sf /usr/share/zoneinfo/Asia/Tokyo /etc/localtime && \
    mkdir /usr/lib/x86_64-linux-gnu/include && \
    ln -s /usr/include/x86_64-linux-gnu/cudnn_v7.h /usr/lib/x86_64-linux-gnu/include/cudnn.h && \
    ldconfig && \
    apt-get clean && \
    apt-get autoremove && \
    rm -rf /var/lib/apt/lists/* /tmp/* ~/*	

EXPOSE 6016

```